In [7]:
# 데이터 전처리

df = pd.read_csv('../train.csv')
df_raw = df.copy()

# Ex~Po의 등급을 가진 변수들은 모두 숫자로 바꿔줌. NA값은 무조건 0으로 만들어줌

cat_num_ls=['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'SalePrice']

tonum = {"ExterQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "ExterCond": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "BsmtQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "BsmtCond": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "HeatingQC": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "KitchenQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "FireplaceQu": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "GarageQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "GarageCond": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "PoolQC": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "BsmtFinType1" : {'GLQ' : 6, 'ALQ' : 5, 'BLQ': 4,'Rec' : 3,'LwQ' : 2, 'Unf' : 1, np.nan : 0},
         "BsmtFinType2" : {'GLQ' : 6, 'ALQ' : 5, 'BLQ': 4,'Rec' : 3,'LwQ' : 2, 'Unf' : 1, np.nan : 0},
        }

df_raw= df_raw.replace(tonum)

# NA 값 처리

def nan_check(df_raw):
    null_check = df_raw.isnull().sum()
    null_count = pd.DataFrame({'column' : null_check.index, 'NaN' : null_check.values})
    # None 값이 있는 column만 가져옴.
    null_count = null_count.query('NaN > 0')
    return(null_count)

def nan_check2(df_raw):
    null_check = df_raw.isnull().sum()
    null_count = pd.DataFrame({'column' : null_check.index, 'NaN' : null_check.values})
    # None 값이 없는 column만 가져옴.
    null_count = null_count.query('NaN == 0')
    return(null_count)

# nan값이 있는 컬럼 이름을 가진 리스트 (13개)
nan_ls = list(set(nan_check(df_raw)['column']) - set(nan_check2(df_raw)['column']))
nan_ls
# description 직접 보면서 결측값이 뭔지 확인

# NA값이 판단된 카테고리 값에 대해 NA에 N이라는 값을 부여 (=N이라는 클래스를 부여)

fill_na_ls = [['GarageType',
 'MiscFeature',
 'BsmtFinType1',
 'Fence',
 'GarageFinish',
 'BsmtExposure',
 'Alley',
 'BsmtFinType2']
]

for name in fill_na_ls:
    df_raw[name] = df_raw[name].fillna('N')

# 원본 데이터 df
# 전처리 마친 데이터 df_raw

In [8]:
df_raw['TotFlrSF'] = df_raw['1stFlrSF'] + df_raw['2ndFlrSF']

In [56]:
df_all = df_raw.copy()
model_all = sm.OLS.from_formula("np.log(SalePrice) ~ scale(OverallQual) + scale(KitchenQual) + scale(GarageCars) + scale(TotalBsmtSF) + scale(TotFlrSF) + scale(BsmtQual) + scale(YearBuilt) + scale(FireplaceQu) + scale(YearRemodAdd) + scale(HeatingQC) + scale(BsmtFinSF1)", data=df_all) #scale(MasVnrArea) + scale(GrLivArea) + scale(TotRmsAbvGrd) +

result_all = model_all.fit()
print(result_all.summary())

                            OLS Regression Results                            
Dep. Variable:      np.log(SalePrice)   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     690.7
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:41:50   Log-Likelihood:                 606.05
No. Observations:                1460   AIC:                            -1188.
Df Residuals:                    1448   BIC:                            -1125.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              12.0241    

In [57]:
# 제출 결과 만들기

# test데이터 전처리
df_test = pd.read_csv('../test.csv')
df_test = df_test.replace(tonum)
for name in fill_na_ls:
    df_test[name] = df_test[name].fillna('N')
    
df_test['TotFlrSF'] = df_test['1stFlrSF'] + df_test['2ndFlrSF']
df_test = df_test.fillna(method='ffill')


pre = pd.DataFrame(result_all.predict(df_test))
pre['Id']=pre.index
pre = pre.rename(columns={0: 'SalePrice'})
pre['Id'] = range(1461,2920)
pre['SalePrice'] = pre['SalePrice'].apply(lambda x : np.exp(x))
pre.to_csv('submission.csv', index=False)

In [58]:
influence = result_all.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (1460 - 1 - 13)
idx = np.where(cooks_d2 > fox_cr)[0]
df_all2 = df_all.copy()
df_all2 = df_all2.drop(df_all2.index[idx])

In [69]:
model_all2 = sm.OLS.from_formula("np.log(SalePrice) ~ C(MSZoning) + scale(OverallQual) + scale(KitchenQual) + scale(GarageCars) + scale(TotalBsmtSF) + scale(TotFlrSF) + scale(YearBuilt) + scale(FireplaceQu) + scale(YearRemodAdd) + scale(HeatingQC) + scale(BsmtFinSF1)", data=df_all2) #scale(MasVnrArea) + scale(GrLivArea) + scale(TotRmsAbvGrd) +

result_all2 = model_all2.fit()
print(result_all2.summary())

                            OLS Regression Results                            
Dep. Variable:      np.log(SalePrice)   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                     1092.
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        17:12:26   Log-Likelihood:                 1141.6
No. Observations:                1395   AIC:                            -2253.
Df Residuals:                    1380   BIC:                            -2175.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              11.9726    

In [60]:
# 제출 결과 만들기

# test데이터 전처리
df_test = pd.read_csv('../test.csv')
df_test = df_test.replace(tonum)
for name in fill_na_ls:
    df_test[name] = df_test[name].fillna('N')
    
df_test['TotFlrSF'] = df_test['1stFlrSF'] + df_test['2ndFlrSF']
df_test = df_test.fillna(method='ffill')


pre = pd.DataFrame(result_all2.predict(df_test))
pre['Id']=pre.index
pre = pre.rename(columns={0: 'SalePrice'})
pre['Id'] = range(1461,2920)
pre['SalePrice'] = pre['SalePrice'].apply(lambda x : np.exp(x))
pre.to_csv('submission.csv', index=False)

In [25]:
df_test['SaleCondition'].unique()

array(['Normal', 'Partial', 'Abnorml', 'Family', 'Alloca', 'AdjLand'],
      dtype=object)

In [26]:
df_raw['SaleCondition'].unique()

array(['Normal', 'Abnorml', 'Partial', 'AdjLand', 'Alloca', 'Family'],
      dtype=object)

In [63]:
df_raw['OverallQual'].describe()

count    1460.000000
mean        6.099315
std         1.382997
min         1.000000
25%         5.000000
50%         6.000000
75%         7.000000
max        10.000000
Name: OverallQual, dtype: float64

In [67]:
sm.stats.anova_lm(result_all2, typ=2)

,sum_sq,df,F,PR(>F)
scale(OverallQual ** 2),2.969478,1.0,235.060485,4.047612e-49
scale(KitchenQual),0.217114,1.0,17.186516,3.594287e-05
scale(GarageCars),1.534845,1.0,121.496558,3.862681e-27
scale(TotalBsmtSF),1.721327,1.0,136.258231,4.287913e-30
scale(TotFlrSF),12.008661,1.0,950.591753,2.602214e-159
scale(BsmtQual),0.038243,1.0,3.027264,8.209841e-02
scale(YearBuilt),0.898750,1.0,71.143991,8.247094e-17
scale(FireplaceQu),1.044979,1.0,82.719296,3.198783e-19
scale(YearRemodAdd),0.781651,1.0,61.874625,7.325574e-15
scale(HeatingQC),0.232911,1.0,18.436959,1.878637e-05
